# Dataset Construction
This notebook retrieves all Beatles song lyrics from [azlyrics.com](www.azlyrics.com/b/beatles.html), cleans them, and stores them as training and test sets.
## Song Lyric Generation with GPT-2
**Data Mining Final Project**<br>
Khyatee Desai<br>Dec. 19, 2022

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [7]:
# scrape azlyrics beatles page
response = requests.get('http://api.scraperapi.com?api_key=2b16bc9cefddc87cc1b93d1f7848fe0c&url=https://www.azlyrics.com/b/beatles.html')
soup = BeautifulSoup(response.text, 'html.parser')


In [8]:
# locate all div tag links - each link goes to a unique song's lyric page
albums = soup.find_all("div", {"class": "listalbum-item"})
links=[]
for album in albums:
    if album.find('a'):
        links.append('https://www.azlyrics.com/'+album.find('a')['href'])
links[:10] # inspect first few links

['https://www.azlyrics.com//lyrics/beatles/isawherstandingthere.html',
 'https://www.azlyrics.com//lyrics/beatles/misery.html',
 'https://www.azlyrics.com//lyrics/beatles/annagotohim.html',
 'https://www.azlyrics.com//lyrics/beatles/chains.html',
 'https://www.azlyrics.com//lyrics/beatles/boys.html',
 'https://www.azlyrics.com//lyrics/beatles/askmewhy.html',
 'https://www.azlyrics.com//lyrics/beatles/pleasepleaseme.html',
 'https://www.azlyrics.com//lyrics/beatles/lovemedo.html',
 'https://www.azlyrics.com//lyrics/beatles/psiloveyou.html',
 'https://www.azlyrics.com//lyrics/beatles/babyitsyou.html']

In [137]:
cleaned_lyrics=[]
for link in links:
    try:  # scrape the lyrics page of each link and save the lyrics to a list
        response = requests.get('http://api.scraperapi.com?api_key=2b16bc9cefddc87cc1b93d1f7848fe0c&url='+link)
        soup = BeautifulSoup(response.text, 'html.parser')
        lyric_section = soup.find('div',{'class':'col-xs-12 col-lg-8 text-center'})
        lyrics = lyric_section.find_all('div')[5].text
        cleaned = lyrics.replace("\n\r\n","").replace("\n\n"," ").replace("\n"," ")
        cleaned_lyrics.append(cleaned)
    except:
        print(link)
        continue

In [139]:
len(cleaned_lyrics)

416

In [11]:
# save lyrics to a dataframe and parquet file
df = pd.DataFrame(cleaned_lyrics)
df.rename(columns={0:'lyrics'},inplace=True)
# df.to_parquet('beatles_lyrics.parquet')
df

,lyrics
0,"(1,2,3,4!) Well, she was just seventeen You kn..."
1,The world is treating me bad... Misery I'm the...
2,"Anna You come and ask me, girl To set you free..."
3,"Chains, my baby's got me locked up in chains A..."
4,I been told when a boy kiss a girl Take a trip...
...,...
421,"Oh yeah, all right Are you going to be in my d..."
422,"Oh no brother Can you take me, can you take me..."
423,Gnik nus eht semoc ereh Gnik nus eht semoc ere...
424,Happy birthday to you (to you) Happy birthday ...


## Train/ Test Split
Split the song lyrics into training and test set. We do not create a validation set because we will not be performing any hyperparameter tuning.

In [ ]:
df = pd.read_parquet('beatles_lyrics.parquet').drop_duplicates('lyrics')
df.drop(df[df['lyrics'].str.contains('Speech')].index, inplace=True)
test_set = df.sample(34)
train_set = df.drop(test_set.index)

In [ ]:
# test_set.to_parquet('testing_lyrics.parquet')
# train_set.to_parquet('training_lyrics.parquet')
f = open('training_lyrics.txt', 'w')
train_string = ' '.join(list(train_set.lyrics.values))
# f.write(train_string)